In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install pygismeteo
!pip install geopy
!pip install tzwhere
!pip install pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/rubert_large_squad_2")
model = AutoModelForCausalLM.from_pretrained("Den4ikAI/rubert_large_squad_2")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at Den4ikAI/rubert_large_squad_2 were not used when initializing BertLMHeadModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at Den4ikAI/rubert_large_squad_2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.p

In [ ]:
import telebot
import requests
###################
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/nllb-200-distilled-600M-ru-persona-chat")
model = AutoModelWithLMHead.from_pretrained("DeepPavlov/nllb-200-distilled-600M-ru-persona-chat")

###############################
bot = telebot.TeleBot('6218469964:AAFoKlfcw878W8_zJvnNRulMJGnuBaGbZh0')
from pygismeteo import Gismeteo
import geopy
from tzwhere import tzwhere
import datetime
import pytz


@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я бот!\nЯ умею разговаривать.\nНапиши свой запрос в сообщении ниже. \n Погода или время или еще какую байду.')

qa_pipeline = pipeline(
     "question-answering",
     model="Den4ikAI/rubert_large_squad_2",
     tokenizer="Den4ikAI/rubert_large_squad_2")

text = open('my_file.txt', 'r', encoding='utf-8').read()



@bot.message_handler(content_types=['text'])

def repeat_all_messages(message): # Название функции не играет никакой роли
    #text = re.split('[^a-zа-яё]+', message.text, flags=re.IGNORECASE)
    #print(text)
    if "med" in message.text.lower() or " med" in message.text.lower():
        #print(predictions)
        #bot.send_message(message.chat.id, output[0]['generated_text'])
        msg = str(message.text.lower()).replace('med', '')
        predictions = qa_pipeline({
            'context': text,
            'question': msg
        })
        bot.send_message(message.chat.id, predictions['answer'])
    elif "погод" in message.text.lower() or " погод" in message.text.lower():
        bot.send_message(message.chat.id, 'Введи название своего города и я скажу какая погода')
        bot.register_next_step_handler(message, weather_reply)
    elif "врем" in message.text.lower() or " врем" in message.text.lower():
        bot.send_message(message.chat.id, 'Введи название своего города и я скажу время')
        bot.register_next_step_handler(message, time_reply)
    else:
        prompt = message.text
        tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt")
        generated_response = model.generate(tokenized_prompt, max_length=50)
        generated_response_text = tokenizer.decode(generated_response[0], skip_special_tokens=True)
        bot.send_message(message.chat.id, str(generated_response_text))

def weather_reply(message):
    city = message.text
    bot.send_message(message.chat.id, str(get_weather(city)))
    #bot.send_message(message.chat.id, 'Выберите тему', reply_markup=get_buttons())

def time_reply(message):
    city = message.text
    bot.send_message(message.chat.id, str(get_time(city)))
    #bot.send_message(message.chat.id, 'Выберите тему', reply_markup=get_buttons())

def get_weather(city):
    gismeteo = Gismeteo()
    try:
        search_results = gismeteo.search.by_query(city)
        city_id = search_results[0].id
        current = gismeteo.current.by_id(city_id)
        return "Погода в " + city + " " + str(current.temperature.air.c) + "С градусов"
    except Exception as ex:
        print(ex)
        return "Город " + city + " не найден"

def get_time(city):
    geo = geopy.geocoders.Nominatim(user_agent="SuperMon_Bot")
    location = geo.geocode(city)
    try:
        tzw = tzwhere.tzwhere()
        timezone_str = tzw.tzNameAt(location.latitude, location.longitude)
        tz = pytz.timezone(timezone_str)
        vremya_goroda = datetime.datetime.now(tz)
        return "Время в городе " + city + " " + str(str(vremya_goroda.hour) + ":" + str(vremya_goroda.minute)) + ""
    except Exception as ex:
        print(ex)
        return "Город " + city + " не найден"

    
if __name__ == '__main__':
     bot.infinity_polling()

2023-03-03 20:52:04,848 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-03-03 20:52:04,851 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
